# Case study for Robot Vaccine: SP

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import itertools
import matplotlib.pylab as plt
from matplotlib import rc
rc("text", usetex=True)
rc("font", family="serif")

import run_robot
import prepare_data
from importlib import reload
reload(prepare_data)
reload(run_robot)

In [ ]:
# Define the basic data for the case studies

# Mean incubation time
tinc = 5.2

# Mean inffectious time
tinf = 2.9

# Basic reproduction number
basic_rep = 2.5

# Lockdown level
lock_level = 1.0

# Simulation horizon
# A little more than a year when thevaccine should be here
ndays = 7*52

# Mean time in ICU
time_icu = 7

# Transmission/infection effect of the vaccine 
r_atten = [1.0, 0.24, 0.176]

# Symptoms effect of the vaccine
icu_atten = [1.0, 1.0, 1.0]

# R0 factor for each population
r0pop = [1.0, 1.3, 1.0, 1.0]

# Max dose profile
max_doses = 0.015*np.ones(ndays)
max_doses[0:30] = 0.001
max_doses[3:150] = 0.005

# Define basic paramters
basic_prm = prepare_data.save_basic_parameters(tinc=tinc, tinf=tinf, rep=basic_rep, min_level=lock_level,
    time_icu=time_icu, alternate=0, window=14, ndays=ndays)

pre_cities_data = {
    "S1": [0.685], 
    "E1": [0.01], 
    "I1": [0.006], 
    "R1": [0.299], 
    "population": [44639899], 
    "icu_capacity": [0.000175], 
    "start_date": [pd.to_datetime("2020-2-1")]
}
cities_data = pd.DataFrame(pre_cities_data, index=["SP"])
cities_data

In [ ]:
# Create a target matrix (max infected level)
ncities, ndays = len(cities_data.index), int(basic_prm["ndays"])
target = 0.8*np.ones((ncities, ndays))
target = prepare_data.save_target(cities_data, target)

# Use a forcedif that releases the cities in the end
force_dif = np.zeros((ncities, ndays))

In [ ]:
# Add information on the time series that estimate the need of ICUs
# We are using the time series adjusted considering that the mean ICU stay is 7 days (which lead to larger ICU capacity).

# Define the time series to estimate ICU demand
ts_sp = np.array([0.009, 0.015, 0.492, 0.0, 0.0, 0.0, 0.083, 0.011, 0.011])

ts_drs = ts_sp*np.ones((len(cities_data), len(ts_sp)))
ts_drs = pd.DataFrame(data=ts_drs, index=cities_data.index, columns=[
    "rho_min", "rho_max", "intercept", "trend", "phi_1", "phi_2", "sigma_omega", "state0", "state_less_1"
])
ts_drs["confidence"] = 0.9
ts_drs["time_icu"] = time_icu
cities_data = pd.concat([cities_data, ts_drs], axis=1)
cities_data

In [ ]:
pd.set_option("display.width", 120)

# Simple function to run a test and save results
def run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0factor, r_atten, icu_atten, max_doses, verbosity=1):
    run_robot.prepare_optimization(basic_prm, cities_data, M, target, hammer_data, force_dif,
        r0factor, r_atten, icu_atten, max_doses, verbosity=verbosity)
    run_robot.optimize_and_show_results(basic_prm, figure_file, result_file, cities_data, target, verbosity=verbosity)

In [ ]:
# Define mobility matrix.
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs="data/report_drs_mobility.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
    incr_all=True, verbosity=1)
M

In [ ]:
def agregate(solution, labels):
    values = solution.loc[labels[0]].sum()
    for i in range(1, len(labels)):
        values += solution.loc[labels[i]].sum()
    return values

## Without vacination 

In [ ]:
# Obs: Set max_doses to 0 in robot_dance.jl before running this cell.

reload(run_robot)
print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
base_name = f"results/without_vaccine_profile"
result_file = base_name + ".csv"
run_a_test(basic_prm, result_file, "", cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, np.zeros(ndays));

In [ ]:
solution = pd.read_csv(result_file, index_col=[0, 1, 2])

i = agregate(solution, ["i", "ir"])
rt = solution.loc["rt", -1, -1]
run_robot.plot_result("SP", basic_prm, rt, i, hammer_data["duration"], start_date="2020-02-01")
plt.savefig(base_name + "_profile.png")

Without vacination the epidemic will last until mid September what it recedes due to herd immunity. The social distancing measures are gradually relaxed.

## With vacination blocking infection

In [ ]:
reload(run_robot)

r_atten = [1.0, 0.24, 0.176]
icu_atten = [1.0, 1.0, 1.0]
max_doses[0:30] = 0.001
print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
base_name = f"results/vaccine_infection"
result_file = base_name + ".csv"
run_a_test(basic_prm, result_file, "", cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, max_doses);

In [ ]:
solution = pd.read_csv(result_file, index_col=[0, 1, 2])
i = agregate(solution, ["i", "ir"])
rt = solution.loc["rt", -1, -1]
run_robot.plot_result("SP", basic_prm, rt, i, hammer_data["duration"], start_date="2020-02-01")
plt.savefig(base_name + "_profile.png")

With vaccination the epidemic ends by June. The need for social distancing end in the start of May, from there on the control is based on vaccines only.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

bins = [[0, 19], [20, 49], [50, 64], [65, 200]]
colors = ["C0", "C1", "C2", "C3"]

def retrieve_vaccination_profile(sol, pop=0, dose=0):
    sol = sol.loc[:, pop, :]
    seir = sol.loc["s", :] + sol.loc["e", :] + sol.loc["i", :] + sol.loc["r", :]
    applied = sol.loc["v", dose]*seir.loc[dose]
    return applied[14:]

In [ ]:
for i in range(len(bins)):
    retrieve_vaccination_profile(solution, i, 0).plot(lw=3, color=colors[i], alpha=0.7, label=f"{bins[i]}, dose 1")
    retrieve_vaccination_profile(solution, i, 1).plot(lw=3, alpha=0.7, color=colors[i], ls="--", label=f"{bins[i]}, dose 2")
ticks = range(0, solution.loc["s", 0, 0, :].shape[1], 28)
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Vaccination")
plt.savefig(base_name + "_vaccines.png")

In [ ]:
total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
for i in range(1, len(bins)):
    total_dose1 += retrieve_vaccination_profile(solution, i, 0)
total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
for i in range(1, len(bins)):
    total_dose2 += retrieve_vaccination_profile(solution, i, 1)
total_dose1.plot(lw=3, alpha=0.7, label="Total dose 1")
total_dose2.plot(lw=3, ls="--", alpha=0.7, label="Total dose 2")
both = total_dose1 + total_dose2 
both.plot(lw=3, alpha=0.3, label="Both doses")
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Total daily vaccination");
print("Total doses", sum(both))
plt.savefig(base_name + "_total_vaccines.png")

In this case the second dose is postponed to the 84th day (the maximum allowed). The first group to be vaccinated is 65+, followed by a periodo anternating between 20+ (that has a higher R0) and 50+. 65+ gets the second dose before the epidemic recedes and the epidemic is controled follwed again by a mex of 50+ adn 20+. After that the vaccination can go slower. 

## With vacination blocking bad symptoms

In [ ]:
reload(run_robot)

icu_atten = [1.0, 0.24, 0.176]
r_atten = [1.0, 1.0, 1.0]
max_doses[0:30] = 0.001
print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
base_name = f"results/vaccine_symptoms"
result_file = base_name + ".csv"
run_a_test(basic_prm, result_file, "", cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, max_doses);

In [ ]:
solution = pd.read_csv(result_file, index_col=[0, 1, 2])
i = agregate(solution, ["i", "ir"])
rt = solution.loc["rt", -1, -1]
run_robot.plot_result("SP", basic_prm, rt, i, hammer_data["duration"], start_date="2020-02-01")
plt.savefig(base_name + "_profile.png")

The epidemic is also controlled sooner, but using more social distancing (28 extra days in low) than when the vaccine blocks transmission. 

In [ ]:
for i in range(len(bins)):
    retrieve_vaccination_profile(solution, i, 0).plot(lw=3, color=colors[i], alpha=0.7, label=f"{bins[i]}, dose 1")
    retrieve_vaccination_profile(solution, i, 1).plot(lw=3, alpha=0.7, color=colors[i], ls="--", label=f"{bins[i]}, dose 2")
ticks = range(0, solution.loc["s", 0, 0, :].shape[1], 28)
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Vaccination")
plt.savefig(base_name + "_vaccines.png")

In [ ]:
total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
for i in range(1, len(bins)):
    total_dose1 += retrieve_vaccination_profile(solution, i, 0)
total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
for i in range(1, len(bins)):
    total_dose2 += retrieve_vaccination_profile(solution, i, 1)
total_dose1.plot(lw=3, alpha=0.7, label="Total dose 1")
total_dose2.plot(lw=3, ls="--", alpha=0.7, label="Total dose 2")
both = total_dose1 + total_dose2 
both.plot(lw=3, alpha=0.3, label="Both doses")
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Total daily vaccination")
plt.savefig(base_name + "_total_vaccines.png")
print("Total doses", sum(both))

Again the second dose is postponed to something close to the 84th day, again the 65+ gets its first doses, followed 50+ only and  after tht 20+. In the middle of the first doses of 20+, the elder group starts receiving it second dose followed by second doses to 50+. After that there is a mix of first and second doses to 20+ and the younger group.

## With vacination blocking infection but with 0.5% vaccines available since the start

In [ ]:
reload(run_robot)

r_atten = [1.0, 0.24, 0.176]
icu_atten = [1.0, 1.0, 1.0]
max_doses[0:30] = 0.005
print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
base_name = f"results/vaccine_infection_more_doses"
result_file = base_name + ".csv"
run_a_test(basic_prm, result_file, "", cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, max_doses);

In [ ]:
solution = pd.read_csv(result_file, index_col=[0, 1, 2])
i = agregate(solution, ["i", "ir"])
rt = solution.loc["rt", -1, -1]
run_robot.plot_result("SP", basic_prm, rt, i, hammer_data["duration"], start_date="2020-02-01")
plt.savefig(base_name + "_profile.png")

Now the pandemic is controlled much sooner by mid April moderate social distancing mesures only after the initial hammer phase. 

In [ ]:
for i in range(len(bins)):
    retrieve_vaccination_profile(solution, i, 0).plot(lw=3, color=colors[i], alpha=0.7, label=f"{bins[i]}, dose 1")
    retrieve_vaccination_profile(solution, i, 1).plot(lw=3, alpha=0.7, color=colors[i], ls="--", label=f"{bins[i]}, dose 2")
ticks = range(0, solution.loc["s", 0, 0, :].shape[1], 28)
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Vaccination")
plt.savefig(base_name + "_vaccines.png")

In [ ]:
total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
for i in range(1, len(bins)):
    total_dose1 += retrieve_vaccination_profile(solution, i, 0)
total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
for i in range(1, len(bins)):
    total_dose2 += retrieve_vaccination_profile(solution, i, 1)
total_dose1.plot(lw=3, alpha=0.7, label="Total dose 1")
total_dose2.plot(lw=3, ls="--", alpha=0.7, label="Total dose 2")
both = total_dose1 + total_dose2 
both.plot(lw=3, alpha=0.3, label="Both doses")
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Total daily vaccination")
plt.savefig(base_name + "_total_vaccines.png")
print("Total doses", sum(both))

In this case the second dose is postponed to around the 84th day. Note that here **the young group 20+ gets the very first doses** followed by 65+ and a mix between 50+ and 20+. The second shots start with 20_ and 65+ folllowed by 50+.  This indicates indicating that the 20+ higher R0 is relevant and is partially attenuated by these initial doses. Remeber that the vaccine in this case is assumed only to block infection.

## With vacination blocking bad symptoms but with 0.5% vaccines available since the start

In [ ]:
reload(run_robot)

icu_atten = [1.0, 0.24, 0.176]
r_atten = [1.0, 1.0, 1.0]
max_doses[0:30] = 0.005
print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
base_name = f"results/vaccine_symptoms_more_doses"
result_file = base_name + ".csv"
run_a_test(basic_prm, result_file, "", cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, max_doses);

In [ ]:
solution = pd.read_csv(result_file, index_col=[0, 1, 2])
i = agregate(solution, ["i", "ir"])
rt = solution.loc["rt", -1, -1]
run_robot.plot_result("SP", basic_prm, rt, i, hammer_data["duration"], start_date="2020-02-01")
plt.savefig(base_name + "_profile.png")

Again, it takes a longer to control the pandemic if the vaccine only attenuates bad symptoms. There are 42 days of moderate to low social distancing after the initial hammer. 

In [ ]:
for i in range(len(bins)):
    retrieve_vaccination_profile(solution, i, 0).plot(lw=3, color=colors[i], alpha=0.7, label=f"{bins[i]}, dose 1")
    retrieve_vaccination_profile(solution, i, 1).plot(lw=3, alpha=0.7, color=colors[i], ls="--", label=f"{bins[i]}, dose 2")
ticks = range(0, solution.loc["s", 0, 0, :].shape[1], 28)
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Vaccination")
plt.savefig(base_name + "_vaccines.png")

In [ ]:
total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
for i in range(1, len(bins)):
    total_dose1 += retrieve_vaccination_profile(solution, i, 0)
total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
for i in range(1, len(bins)):
    total_dose2 += retrieve_vaccination_profile(solution, i, 1)
total_dose1.plot(lw=3, alpha=0.7, label="Total dose 1")
total_dose2.plot(lw=3, ls="--", alpha=0.7, label="Total dose 2")
both = total_dose1 + total_dose2 
both.plot(lw=3, alpha=0.3, label="Both doses")
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Total daily vaccination")
plt.savefig(base_name + "_total_vaccines.png")
print("Total doses", sum(both))

In this casem the second dose is also delayed up to the last day (day 84). Since the vaccine can only alliviate symptoms it is more important to vaccinate the elder population first and this is very clear in the vaccinatin profile where the first and second doses come in order of age.